In [23]:
from conf import download_path, db_conn, kite as kite_conf
import polars as pl
from src.scans.swing_scan import basic_filter, add_basic_indicators
from datetime import datetime
from src.utils import setup_logger
import logging

setup_logger()

logger = logging.getLogger(__name__)

In [24]:
END_DATE = datetime(2025, 12, 12)

In [25]:
scan_symbol_list = (
    pl.scan_parquet(download_path / "adr_stocks.parquet")
    .collect()
    .get_column("symbol")
    .to_list()
)

logger.info(f"Stocks in the Scan List {len(scan_symbol_list)}")

query = f"""
select * 
from {kite_conf["hist_table_name"]}
where symbol in {tuple(scan_symbol_list)}
"""
data = pl.read_database_uri(query=query, uri=db_conn)

basic_stock_list = basic_filter(
    data=data, symbol_list=scan_symbol_list, scan_date=END_DATE
)
query = f"""
select * 
from {kite_conf["hist_table_name"]}
where symbol in {tuple(basic_stock_list)}
"""
data = pl.read_database_uri(query=query, uri=db_conn)

2025-12-14 00:28:16 | INFO | __main__ | <module> | Stocks in the Scan List 498
2025-12-14 00:28:16 | INFO | src.scans.swing_scan | basic_filter | Number of stocks in symbol list: 498
2025-12-14 00:28:16 | INFO | src.scans.swing_scan | basic_filter | Symbols after basic filter: 161


# Reddit Comment

In [26]:
# WINDOW_SIZE = 10

# (
#     data.lazy()
#     .with_columns(
#         # Shift Columns
#         [
#             pl.col(col)
#             .shift(i)
#             .over(partition_by="symbol", order_by="timestamp", descending=False)
#             .alias(f"{col}_prev_{i}")
#             for col in ["close", "timestamp"]
#             for i in [1]
#         ]
#     )
#     # Calculate Ranges
#     .with_columns(
#         (
#             (pl.max_horizontal(pl.col("close") - pl.col("close_prev_1")))
#             - pl.min_horizontal(pl.col("low") - pl.col("close_prev_1"))
#         ).alias("small_range"),
#         (
#             pl.col("close").rolling_max(window_size=WINDOW_SIZE)
#             - pl.col("low").rolling_min(window_size=WINDOW_SIZE)
#         )
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("large_range"),
#     )
#     .with_columns(
#         (
#             (
#                 pl.col("small_range").rolling_sum(window_size=WINDOW_SIZE)
#                 / pl.col("large_range")
#             ).log()
#             / pl.lit(10).log()
#         )
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("R")
#     )
#     .with_columns(
#         ((pl.col("R") > 0.6).cast(pl.Int64).rolling_sum(window_size=5) >= 5)
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("scan")
#     )
#     .filter(pl.col("scan") == True)
#     .filter(pl.col("timestamp") == datetime(2025, 9, 4))
# ).collect()

# PullBack Exp

In [49]:
NEAR_PCT = 0.01

comparisons = [
    (pl.col(f"mid_prev_{i}")) <= pl.col(f"mid_prev_{i + 1}") for i in range(1, 10)
]
df = add_basic_indicators(data=data)
pullback_filter = (
    df.lazy()
    .with_columns([pl.mean_horizontal(("open", "close")).round(2).alias("mid_prev_0")])
    .with_columns(
        [
            pl.col("mid_prev_0")
            .shift(i)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias(f"mid_prev_{i}")
            for i in range(1, 11)
        ]
        + [
            (
                ((pl.col("mid_prev_0") - pl.col(col)).abs() / pl.col(col)) <= NEAR_PCT
            ).alias(f"near_{col}")
            for col in ["close_ema_9", "close_ema_21", "close_sma_50"]
        ]
    )
    .with_columns(
        pl.sum_horizontal(
            [pl.when(cond).then(1).otherwise(0) for cond in comparisons]
        ).alias("mid_down_count")
    )
    .filter(
        (
            (pl.col("near_close_ema_9") == True)
            | (pl.col("near_close_ema_21") == True)
            | (pl.col("near_close_sma_50") == True)
        )
        & (pl.col("mid_down_count") > 2)
        & (pl.col("timestamp") == END_DATE)
        & (pl.col("adr_pct_20") >= 3.5)
        & (pl.col("rvol_pct") < 50)
    )
    .sort(["rvol_pct", "adr_pct_20"], descending=[False, True])
    .with_row_index(name="rank", offset=1)
).collect()

In [50]:
pullback_filter

rank,symbol,timestamp,open,high,low,close,volume,close_sma_50,close_ema_9,close_ema_21,volume_sma_20,volume_sma_50,day_range,adr_pct_20,rvol_pct,mid_prev_0,mid_prev_1,mid_prev_2,mid_prev_3,mid_prev_4,mid_prev_5,mid_prev_6,mid_prev_7,mid_prev_8,mid_prev_9,mid_prev_10,near_close_ema_9,near_close_ema_21,near_close_sma_50,mid_down_count
u32,str,date,f64,f64,f64,f64,i64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool,bool,i32
1,"""AGIIL""",2025-12-12,267.4,269.8,263.1,263.95,78746,266.31,268.18,268.24,449738,838240,1.025466,4.16,9.0,265.67,266.45,267.82,269.45,273.35,272.38,268.77,271.52,270.82,272.2,273.15,true,true,true,6
2,"""INDIGOPNTS""",2025-12-12,1220.9,1240.7,1220.9,1233.9,25446,1147.7,1240.09,1233.97,107429,223916,1.016218,3.69,11.0,1227.4,1225.05,1218.3,1202.35,1221.4,1267.8,1292.8,1284.2,1281.05,1291.4,1276.43,false,true,false,4
3,"""UNIPARTS""",2025-12-12,480.0,491.0,480.0,489.4,52351,479.79,484.46,485.04,146258,439597,1.022917,3.53,12.0,484.7,482.4,481.78,474.9,467.5,482.8,489.5,495.38,499.4,498.88,494.62,true,true,false,4
4,"""PRECWIRE""",2025-12-12,238.6,240.15,234.45,235.9,160392,229.79,238.72,240.34,482924,1275729,1.024312,4.41,13.0,237.25,236.62,235.02,234.98,236.12,236.65,238.77,245.5,246.88,251.6,257.01,true,false,false,7
5,"""AHLUCONT""",2025-12-12,974.3,980.65,952.0,958.7,21258,957.52,976.31,974.31,359423,165803,1.030095,3.93,13.0,966.5,975.7,977.8,959.7,974.62,996.72,1001.65,997.25,985.62,993.4,992.12,false,true,true,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
37,"""ECOSMOBLTY""",2025-12-12,227.56,231.32,225.52,229.82,187254,226.27,230.62,231.75,307530,414157,1.025718,4.87,45.0,228.69,228.18,228.12,225.53,223.99,230.92,235.51,235.92,239.14,242.44,243.33,true,false,false,6
38,"""NOIDATOLL""",2025-12-12,4.68,4.68,4.49,4.52,63996,4.37,4.57,4.53,153664,135593,1.042316,5.2,47.0,4.6,4.67,4.54,4.46,4.69,4.58,4.57,4.89,5.07,4.97,4.68,true,false,false,3
39,"""ATALREAL""",2025-12-12,24.41,24.46,23.99,24.16,1742882,23.85,24.04,23.89,3340058,3732767,1.019591,4.65,47.0,24.28,24.51,24.22,23.94,24.32,24.08,23.72,23.32,23.38,23.1,23.41,true,false,false,3


# AMIBROKER

In [ ]:
timeframe = 252
vol_tf = 50
base_lower_limit = 0.6
pivot_length = 5
pv_limit = 0.1

ami_scan_stocks = (
    (
        data.lazy()
        .with_columns(
            # 52 week high calculation
            pl.col("close")
            .rolling_max(window_size=timeframe)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("52_week_high"),
            # volume sma calculation
            pl.col("volume")
            .rolling_mean(window_size=vol_tf)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias(f"volume_sma_{vol_tf}"),
            # pivot high calculation
            pl.col("high")
            .rolling_max(window_size=pivot_length)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_high"),
            # pivot low calculation
            pl.col("low")
            .rolling_min(window_size=pivot_length)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_low"),
            # pivot start high
            pl.col("high")
            .shift(pivot_length - 1)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_start_high"),
        )
        .with_columns(
            # pivot width
            ((pl.col("pivot_high") - pl.col("pivot_low")) / pl.col("close")).alias(
                "pivot_width"
            )
        )
        .with_columns(
            # find pivot
            (
                (pl.col("pivot_width") < pv_limit)
                & (pl.col("pivot_high") == pl.col("pivot_start_high"))
            ).alias("is_pivot"),
            # volume dry up
            pl.all_horizontal(
                [
                    pl.col("volume").shift(i) < pl.col(f"volume_sma_{vol_tf}").shift(i)
                    for i in range(pivot_length)
                ]
            )
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("vol_dry_up"),
            # near 52 week high
            (
                (pl.col("close") < pl.col("52_week_high"))
                & (pl.col("close") > base_lower_limit * pl.col("52_week_high"))
            ).alias("near_high"),
        )
        .filter(
            pl.col("near_high") & pl.col("is_pivot") & pl.col("vol_dry_up")
            # & pl.col("vol_decreasing")
        )
    )
    .collect()
    .filter(pl.col("timestamp") == END_DATE)
    .sort("symbol")
    .get_column("symbol")
    .to_list()
)

ami_final_stocks = basic_filter(data=data, symbol_list=ami_scan_stocks, scan_date=END_DATE)
ami_final_stocks